In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from scipy import stats as st
from sklearn.model_selection import RandomizedSearchCV as rms
import math
import time
from sklearn.preprocessing import LabelEncoder
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.linear_model import LogisticRegression as lgr
from sklearn import metrics
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.feature_selection import RFECV
import sklearn.metrics as mt
import random
import ieee_base_models as bm
import ieee_feature_selection_and_scaling as fs

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
total_start_time = time.time()
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
def cat_convert(df):
    #categorical
    df_col_group = {}
    df_col_group['card_cols'] = [c for c in df.columns if 'card' in c] 
    df_col_group['id_cols'] = [c for c in df.columns if 'id_' in c]
    df_col_group['addres_cols'] = [c for c in df.columns if 'addr' in c]
    df_col_group['M_cols'] = list(df.filter(regex='\AM[0-9]+\Z').columns) # M# columns
    df_col_group['V_cols'] = list(df.filter(regex='\AV[0-9]+\Z').columns) # V# columns
    df_col_group['email_cols'] = list(df.filter(regex='emaildomain\Z').columns) # P_ e d_ emaildomain columns
    df_col_group['device_cols'] = list(df.filter(regex='\ADevice').columns) # Device columns
    df_col_group['ProductCD'] = ['ProductCD'] 
    for group in df_col_group:
        for column in df_col_group[group]:
            #print(column,df[column].dtype)
            #df[column] = df[column].astype('object')
            df[column] = df[column].astype('category') 
    #numeric
    df_col_group['C_cols'] = list(df.filter(regex='\AC[0-9]+\Z').columns) # C# columns
    df_col_group['D_cols'] = list(df.filter(regex='\AD[0-9]+\Z').columns) # D# columns
    df_col_group['dist_cols'] = list(df.filter(regex='\Adist[0-9]+\Z').columns) # dist# columns
    df_col_group['TransactionAmt'] = ['TransactionAmt'] 
    #df_col_group['TransactionDT'] = ['TransactionDT'] 
    df_col_group
    
    return df, df_col_group

## Preprocessing

### Importation

In [ ]:
start_time = time.time()
train = pd.read_csv("/kaggle/input/preprocessing-lucas/train.csv", index_col=0)
print("--- %.2f minutes ---" % ((time.time() - start_time)/60))

### Changing types

In [ ]:
start_time = time.time()
train = train.fillna(-999)
train = cat_convert(train)[0]
train.loc[:, "addr1"] = train.loc[:, "addr1"].astype("str")
train.loc[:, "addr2"] = train.loc[:, "addr2"].astype("str")
train.loc[:, "card3"] = train.loc[:, "card3"].astype("str")
train.loc[:, "card5"] = train.loc[:, "card5"].astype("str")
print("--- %.2f minutes ---" % ((time.time() - start_time)/60))

### Label Encoding

In [ ]:
start_time = time.time()
train = fs.impute(train, 0.9)
encoding ={}
for feature in train[train.columns[train.dtypes=='category']].columns:
    encoding[feature] = LabelEncoder().fit(list(train[feature].values)) #(list(test)+list(train))
    
for feature in encoding:
    train[feature] = encoding[feature].transform(list(train[feature].values))
print("--- %.2f minutes ---" % ((time.time() - start_time)/60))

### Feature Engineering

In [ ]:
start_time = time.time()
card_id = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_transaction.csv", usecols=["card1", "card2", "card3", "card4", "card5", "card6"])
card_id = pd.DataFrame([str(s) for s in np.stack(card_id.loc[:, ["card1", "card2", "card3", "card4", "card5", "card6"]].values)])

train.loc[:, "card"] = card_id.loc[:, 0]

train_sum = train.drop(["TransactionDT", "TransactionID", "isFraud"], axis=1).groupby("card").agg("sum")
train_sum.columns = "sum_" + train_sum.columns
train_std = train.drop(["TransactionDT", "TransactionID", "isFraud"], axis=1).groupby("card").agg("std")
train_std.columns = "std_" + train_std.columns

train = train.join(train_sum, on="card")
train = train.join(train_std, on="card")

train = train.drop("card", axis=1)
print("--- %.2f minutes ---" % ((time.time() - start_time)/60))

### Feature Selection
#### Filtering

In [ ]:
start_time = time.time()
train = train.replace([np.inf, -np.inf], np.nan)
train = pd.get_dummies(train, drop_first=True)
train = fs.feat_selection(train, 0.9, 1, 0.0005)
scaled_list = fs.scaling(train.drop(["TransactionID"], axis=1))
X_final = scaled_list[0]
cols = scaled_list[1]
scaler = scaled_list[2]
del scaled_list
print("--- %.2f minutes ---" % ((time.time() - start_time)/60))

#### Wrapping

In [ ]:
start_time = time.time()
X_final.loc[:, "TransactionID"] = 0
irrel_feat = fs.wrapper(X_final, 25, 1)
print("--- %.2f minutes ---" % ((time.time() - start_time)/60))

In [ ]:
#start_time = time.time()
#y = train.isFraud
#est = xgb.XGBClassifier(subsample=0.6,n_estimators=10,min_child_weight=2,max_depth=8,learning_rate=0.05,gamma = 15,colsample_bytree=1,eval_metric='auc',objective ="binary:logistic",max_bin=16,random_state=1)
#print("Selection 1")
#fsel = RFECV(est, step=20, cv=15, n_jobs=-1, scoring="roc_auc", min_features_to_select=int(len(train.columns)*0.95))
#fsel = fsel.fit(X_final.drop(["n_frauds", "isFraud"], axis=1), y)
#important_feat = list(X_final.drop(["n_frauds", "isFraud"], axis=1).loc[:, fsel.ranking_<=1].columns) + ["isFraud", "TransactionID"]
#print("--- %.2f minutes ---" % ((time.time() - start_time)/60))

In [ ]:
print("Dataset has", len(X_final.columns) - len(irrel_feat), "columns.")

### Validation

In [ ]:
start_time = time.time()
models = bm.validate(X_final.drop(irrel_feat, axis=1), 10)
print("--- %.2fs minutes ---" % ((time.time() - start_time)/60))

### Importances

In [ ]:
models["XGB"][0].fit(X_final.drop(["isFraud", "TransactionID"] + irrel_feat, axis=1), X_final.isFraud)
importances = models["XGB"][0].get_booster().get_score(importance_type="weight")
importances = pd.DataFrame({"Weight": list(importances.values())}, index=list(importances))
print(importances.sort_values("Weight", ascending=False))
xgb.plot_importance(models["XGB"][0])

### Export Train

In [ ]:
X_final.drop(irrel_feat, axis=1).to_csv("train_final.csv")

In [ ]:
print("--- %.2fs minutes total time ---" % ((time.time() - total_start_time)/60))